In [4]:
import pyrealsense2 as rs

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)


In [ ]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# Load the pre-trained Faster R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()


In [6]:
import cv2
import torch
import numpy as np

while True:
    # Wait for a new frame
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    if not color_frame:
        continue

    # Convert the color frame to a numpy array
    color_image = np.asanyarray(color_frame.get_data())

    # Preprocess the image
    image_tensor = torch.from_numpy(color_image)
    image_tensor = image_tensor.permute(2, 0, 1).float()
    image_tensor /= 255.0
    image_tensor = image_tensor.unsqueeze(0)

    # Perform object detection
    with torch.no_grad():
        predictions = model(image_tensor)
    
    # Display the bounding boxes and labels
    boxes = predictions[0]['boxes'].detach().numpy()
    labels = predictions[0]['labels'].detach().numpy()
    
    for box, label in zip(boxes, labels):
        x1, y1, x2, y2 = box.astype(int)
        cv2.rectangle(color_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(color_image, str(label), (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Object Detection', color_image)
    
    # Exit if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Clean up
cv2.destroyAllWindows()
pipeline.stop()


KeyboardInterrupt: 

: 